In [24]:
import numpy as np
import pandas as pd
import os 
import win32com.client
from datetime import datetime, date, timedelta

In [25]:
x = datetime.now().hour
today = date.today()
#yesterday = today - timedelta(days=1)
print("The current time is", x)
print(today)
#print(yesterday)

The current time is 9
2024-12-09
2024-12-08


In [26]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.GetDefaultFolder(6)
messages = inbox.Items
for message in messages:
    if message.SentOn.date() == today and message.Class == 43:
        print(message.Subject)
        if ("Job VMI_JU_PO_BL_QTY" in message.Subject) or ("Job VMI_JU_STOCK LEVEL" in message.Subject):
            attachments = message.Attachments
            for attachment in attachments:
                attachment.SaveAsFile(os.path.join(r'\\Siwdsntv002\sg_psc_sg1_pl_08_control_whse\Past Interns\Ryan\Eriks', attachment.FileName))

Job ME5A PR DAILY REPORT, Step 1
Job YIFR_ENG SPARE INCOMING_V1, Step 1
You have 4 actions due in next 30 days
Ryan, respond to your 16 new coworkers, 50 group updates and 56 unread notifications


In [9]:
directory = r'\\Siwdsntv002\sg_psc_sg1_pl_08_control_whse\Past Interns\Ryan\Eriks'
i = 0
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if x < 12:
        if filename == 'Job VMI_JU_STOCK LEVEL, Step 1.htm':
            stock_df = pd.read_html(file_path, header=0)[2]
            i += 1
        elif filename == 'Job VMI_JU_PO_BL_QTY, Step 1.htm':
            po_df = pd.read_html(file_path, header=0)[0]
            i += 1
    elif x > 12:
        if filename == 'Job VMI_JU_STOCK LEVEL AFTERNOON, Step 1.htm':
            stock_df = pd.read_html(file_path, header=0)[2]
            i += 1
        elif filename == 'Job VMI_JU_PO_BL_QTY AFTERNOON, Step 1.htm':
            po_df = pd.read_html(file_path, header=0)[0]
            i += 1
print(i)
        
print(stock_df.head())
print(po_df.head())

2
   Plnt        Material                     Material Description Typ  \
0  SG93  50000400000397     ATG MAXIFLEX KNITWRIST GLOVE SIZE 10  Z1   
1  SG93  50000400000398     ATG MAXIFLEX KNITWRIST GLOVE SIZE 09  Z1   
2  SG93  50000400000399     ATG MAXIFLEX KNITWRIST GLOVE SIZE 08  Z1   
3  SG93  50000400000400     ATG MAXIFLEX KNITWRIST GLOVE SIZE 07  Z1   
4  SG93  50000400201967  CLEANING-WIPERS 9X9 NON-WOVEN CELIULOSE  Z1   

   Reorder Pt  Fixed lot  Unrestr.  PGr  MvAvgPrice  \
0        72.0      144.0     252.0  S15        3.14   
1        72.0      144.0     204.0  S15        3.59   
2        72.0      144.0     252.0  S15        3.07   
3        12.0       12.0      57.0  S15        3.11   
4        60.0       60.0     141.0  S13       13.99   

              Mfr Part Number  ...  Stor.  RShLi  SLife Period Ind  \
0  MAXIFLEX KNITWRIST SIZE 10  ...      0      0      0          D   
1  MAXIFLEX KNITWRIST SIZE 09  ...      0      0      0          D   
2  MAXIFLEX KNITWRIST SI

In [10]:
#stock_df.columns.values.tolist()
#print(po_df.columns.values.tolist())

In [11]:
stock_col = ["Material", "Material\xa0Description", "Reorder\xa0Pt", "Fixed\xa0lot", "Unrestr." ,"BUn","SLoc"]
stock_col_rename = {"Material":'Material', "Material\xa0Description" : "Material_Description", "Reorder\xa0Pt" : "VMI Drop To (Min level)", "Fixed\xa0lot" : "VMI Top Up", "Unrestr." : "GSK Stock Level","BUn":"Base Unit of Measure"}
stock_df = stock_df[stock_col]
stock_df = stock_df.rename(columns=stock_col_rename)
stock_df = stock_df[stock_df["GSK Stock Level"] != 0]
mask = (
    ((stock_df['Material'] == 55000000020911) & (stock_df['SLoc'] != 5002)) |
    ((stock_df['Material'] == 55000000020913) & (stock_df['SLoc'] != 5002)) |
    ((stock_df['Material'] == 55000000020912) & (stock_df['SLoc'] != 5002))
)

stock_df = stock_df[~mask]
print(stock_df.head())

         Material                     Material_Description  \
0  50000400000397     ATG MAXIFLEX KNITWRIST GLOVE SIZE 10   
1  50000400000398     ATG MAXIFLEX KNITWRIST GLOVE SIZE 09   
2  50000400000399     ATG MAXIFLEX KNITWRIST GLOVE SIZE 08   
3  50000400000400     ATG MAXIFLEX KNITWRIST GLOVE SIZE 07   
4  50000400201967  CLEANING-WIPERS 9X9 NON-WOVEN CELIULOSE   

   VMI Drop To (Min level)  VMI Top Up  GSK Stock Level Base Unit of Measure  \
0                     72.0       144.0            252.0                   EA   
1                     72.0       144.0            204.0                   EA   
2                     72.0       144.0            252.0                   EA   
3                     12.0        12.0             57.0                   EA   
4                     60.0        60.0            141.0                  PAC   

   SLoc  
0  5001  
1  5001  
2  5001  
3  5001  
4  5001  


In [12]:
po_col = ["Material",'Pur.\xa0Doc.','To\xa0be\xa0del.']
po_col_rename = {"Material":"Material",'Pur.\xa0Doc.':"VMI_Blanket_PO",'To\xa0be\xa0del.':"Balance_VMI_PO_Quanity"}
po_df = po_df[po_col]
po_df = po_df.rename(columns=po_col_rename)
print(po_df.head())

         Material  VMI_Blanket_PO  Balance_VMI_PO_Quanity
0  50000408518017      4503179283                    80.0
1  55000000002000      4503179283                   229.0
2  55000000002001      4503179283                    45.0
3  50000400500326      4503269893                   280.0
4  50000400500328      4503269893                    31.0


In [13]:
po_stock = pd.merge(stock_df,po_df,left_on = "Material",right_on="Material",how= "left")
po_stock[["VMI_Blanket_PO"]].astype(str)
po_stock.head()
po_stock["VMI Safety Stock Level(Max Level)"] = (po_stock["VMI Drop To (Min level)"].fillna(0) + po_stock["VMI Top Up"].fillna(0))
#po_stock["Top Up VMI Qty"] = np.where(po_stock["GSK Stock Level"] <= po_stock["VMI Drop To (Min level)"], po_stock["VMI Top Up"],0)

In [14]:
conditions = [(po_stock["GSK Stock Level"] <= po_stock["VMI Drop To (Min level)"]),(po_stock["GSK Stock Level"] > po_stock["VMI Drop To (Min level)"])]
values = [po_stock["VMI Top Up"],0]
po_stock["Top Up VMI Qty"] = np.select(conditions,values)

In [15]:
po_stock.head()

,Material,Material_Description,VMI Drop To (Min level),VMI Top Up,GSK Stock Level,Base Unit of Measure,SLoc,VMI_Blanket_PO,Balance_VMI_PO_Quanity,VMI Safety Stock Level(Max Level),Top Up VMI Qty
0,50000400000397,ATG MAXIFLEX KNITWRIST GLOVE SIZE 10,72.0,144.0,252.0,EA,5001,4.503320e+09,576.0,216.0,0.0
1,50000400000398,ATG MAXIFLEX KNITWRIST GLOVE SIZE 09,72.0,144.0,204.0,EA,5001,4.503320e+09,104.0,216.0,0.0
2,50000400000399,ATG MAXIFLEX KNITWRIST GLOVE SIZE 08,72.0,144.0,252.0,EA,5001,4.503320e+09,288.0,216.0,0.0
3,50000400000400,ATG MAXIFLEX KNITWRIST GLOVE SIZE 07,12.0,12.0,57.0,EA,5001,4.503320e+09,144.0,24.0,0.0
4,50000400201967,CLEANING-WIPERS 9X9 NON-WOVEN CELIULOSE,60.0,60.0,141.0,PAC,5001,4.503357e+09,150.0,120.0,0.0


In [17]:
excel_name = "VMI.xlsx"
excel_file_path = os.path.join(r'\\Siwdsntv002\sg_psc_sg1_pl_08_control_whse\Past Interns\Ryan\Eriks', excel_name)
with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='w') as writer:
    po_stock.to_excel(writer, sheet_name="VMI", index=False)